<a href="https://colab.research.google.com/github/Achraf-Trabelsi/PFA-NLP/blob/main/LSTM_w_o_pos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/PFA/data/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c feedback-prize-2021
!mkdir /content/data
!unzip -q -n feedback-prize-2021.zip -d /content/data 

Mounted at /content/drive
 77% 25.0M/32.4M [00:00<00:00, 55.0MB/s]
100% 32.4M/32.4M [00:00<00:00, 63.8MB/s]


In [ ]:
import pandas as pd
import numpy as np
import os 
import tensorflow as tf
from tensorflow import keras
from keras import layers
path='/content/data/'
data=pd.read_csv(f'{path}train.csv')
data.head()
data.sort_values('id')
data.head()

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...


In [ ]:
import nltk
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer
nltk.download('averaged_perceptron_tagger')
nltk.download('state_union')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package state_union to /root/nltk_data...
[nltk_data]   Unzipping corpora/state_union.zip.


True

In [ ]:
sample_text = state_union.raw("2006-GWBush.txt")
train_text  = state_union.raw("2005-GWBush.txt") 
custom_sent_tokenizer = PunktSentenceTokenizer(sample_text)

In [ ]:
tokenized = custom_sent_tokenizer.tokenize(sample_text)

In [ ]:
def make_tag_lookup_table():
    labels = list(pd.unique(data['discourse_type'].values))
    labels = ['O'] + labels
    return dict(zip(range(0,len(labels)+1),labels))

mapping = make_tag_lookup_table()
print(mapping)

{0: 'O', 1: 'Lead', 2: 'Position', 3: 'Evidence', 4: 'Claim', 5: 'Concluding Statement', 6: 'Counterclaim', 7: 'Rebuttal'}


In [ ]:
def tagged_text(data,size):
        path='/content/data'
        mapping = make_tag_lookup_table()
        k=0
        df=pd.DataFrame(columns=['id','text','POS tags','Arg tags'])
        ids=pd.unique(data['id'].values)[:]
        for id in ids:
            k=k + 1
            if k % 1000 == 0:
                print(f'{k} \n')
            try:
                f=open(path+'/train/'+id+'.txt','r')
                tagged_list=[]
                text=f.read()
                tokenized = custom_sent_tokenizer.tokenize(text)
                for i in tokenized:
                    words = i.split()
                    tagged = nltk.pos_tag(words)
                    tagged_list.extend(tagged)

                arg_tags= np.zeros(len(text.split()),dtype=float)
                for (key,tag) in mapping.items():
                    for prediction in data.loc[(data['id']==id) & (data['discourse_type']==tag),'predictionstring']:
                        pred_array=list(map(int, prediction.split()))
                        arg_tags[pred_array]=key
                df=df.append({'id':id, 'text': text, 'POS tags': tagged_list,'Arg tags': arg_tags}, ignore_index=True)
            except Exception as e:
                print(str(e))
        return df

In [ ]:
df = tagged_text(data,5000)

1000 

2000 

3000 

4000 

5000 

6000 

7000 

8000 

9000 

10000 

11000 

12000 

13000 

14000 

15000 



In [ ]:
max_seq_length = max(len(x) for x in df['Arg tags'].values)
max_seq_length=800
print(max_seq_length)

500


In [ ]:
def write_train_files(df):
    ids=df['id'].values
    corpus_size=len(ids)
    for  i in range(0,corpus_size):
      id = ids[i]
      tok = [x[0] for x in df['POS tags'][i]]
      pos = [x[1] for x in df['POS tags'][i]]
      arg = df['Arg tags'][i]
      path='data'
      if len(pos) == len(arg):
        l=len(pos)
      else:
        l = min(len(pos),len(arg)) 
      try:
          with open(f'{path}/train_data/{id}.txt', 'a') as the_file:

              for j in range(0,l):
                t = tok[j]
                p = pos[j]
                a = arg[j]
                the_file.write(f'{t} {p} {a}\n')
      except FileNotFoundError as e:
          os.system(f'mkdir {path}/train_data/')
          the_file=open(f'{path}/train_data/{id}.txt', 'x')
          for j in range(0,l):
              t = tok[j]
              p = pos[j]
              a = arg[j]
              the_file.write(f'{t} {p} {a}\n')



In [ ]:
!rm /content/data/train_data/*
write_train_files(df)

rm: cannot remove '/content/data/train_data/*': No such file or directory


In [ ]:
#!cat /content/data/train_data/E05C7F5C1156.txt

In [ ]:
import urllib.request
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q -n "/content/glove.6B.zip" -d "/content/"
emmbed_dict = {}
with open('/content/glove.6B.50d.txt','r') as f:
  for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:],'float32')
    emmbed_dict[word]=vector

--2022-06-02 12:53:03--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-06-02 12:53:04--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-06-02 12:53:04--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
import nltk
from nltk.data import load
nltk.download('tagsets')
tagdict = load('help/tagsets/upenn_tagset.pickle')
tagdict.keys()

[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.


dict_keys(['LS', 'TO', 'VBN', "''", 'WP', 'UH', 'VBG', 'JJ', 'VBZ', '--', 'VBP', 'NN', 'DT', 'PRP', ':', 'WP$', 'NNPS', 'PRP$', 'WDT', '(', ')', '.', ',', '``', '$', 'RB', 'RBR', 'RBS', 'VBD', 'IN', 'FW', 'RP', 'JJR', 'JJS', 'PDT', 'MD', 'VB', 'WRB', 'NNP', 'EX', 'NNS', 'SYM', 'CC', 'CD', 'POS'])

In [ ]:
emmbed_dict['feel'].shape

(50,)

In [ ]:
embeddings_index = dict()
f = open('/content/glove.6B.50d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()

In [ ]:
from numpy.ma.core import empty
import re
import nltk
from nltk.data import load
from keras.preprocessing.text import Tokenizer
import csv


def pos_lookup_table():
  tagdict = load('help/tagsets/upenn_tagset.pickle')
  pos_dict=dict()
  i=0
  pos_dict['0']=0
  for key in tagdict.keys():
    i=i+1
    pos_dict[key]=i

  return pos_dict

def wordNormalize(word):
    word = word.lower()
    word = word.replace("--", "-")
    word = re.sub("\"+", '"', word)
    word = re.sub("[0-9]{4}-[0-9]{2}-[0-9]{2}", '0', word)
    word = re.sub("[0-9]{2}:[0-9]{2}:[0-9]{2}", '0', word)
    word = re.sub("[0-9]{2}:[0-9]{2}", '0', word)
    word = re.sub("[0-9.,]+", '0', word)
    return word
  
def create_dataset(emmbed_dim, max_words):
  try: 
    fi=open(f'/content/data/dataset.csv', 'x')
    writer=csv.writer(fi,delimiter=',',lineterminator='\n')
    writer.writerow(['id','text','pos', 'arg'])
    fi.close()
  except Exception as e:
    os.system('rm /content/data/dataset.csv')
    fi=open(f'/content/data/dataset.csv', 'x')
    writer=csv.writer(fi,delimiter=',',lineterminator='\n')
    writer.writerow(['id','text','pos', 'arg'])
    fi.close()
  
  #dataset=pd.DataFrame(columns=['id','text','POS tags','Arg tags'])
  pos_dict=pos_lookup_table()
  for files in os.listdir('/content/data/train_data'): 
    id = files.strip('.txt')

    word_count=0
    with open(f'/content/data/train_data/{files}','r') as f:
      #docVec=np.empty(shape=(max_words, emmbed_dim))
      doc = str()
      pos_tag_vec=[]
      arg_tag_vec=[]
      for line in f:
        splits = line.strip().split()
        if len(splits) > 1:
          word = splits[0]
          if word_count == max_words :
            break;
          word_count=word_count+1
          wordLower = word.lower()
          wordNorm = wordNormalize(wordLower)

          #wordVec=np.reshape(wordVec,newshape=(emmbed_dim,1)).T
          pos_tag = splits[1]
          try:
            pos_tag_code = pos_dict[pos_tag]
          except Exception as e:
            pos_tag_code = -1
          
          arg_tag = splits[2]
         
          doc = doc + ' ' + wordNorm
          pos_tag_vec.append(pos_tag_code)
          arg_tag_vec.append(arg_tag)

      #docVec = np.delete(docVec,[range(0,max_words)], axis=0)
      #pos_tag_vec = np.delete(pos_tag_vec,range(0,max_words))
      #arg_tag_vec = np.delete(arg_tag_vec,range(0,max_words))

      data = { 
          'id': id,
          'text': doc,
          'POS tags': pos_tag_vec,
          'Arg tags': arg_tag_vec
          }
      
      try: 
        fi = open(f'{path}/dataset.csv', 'a')
        writer = csv.writer(fi, delimiter=',',lineterminator='\n')
        row = [id , doc, ' '.join(map(str,pos_tag_vec)),' '.join(map(str,arg_tag_vec))]
        writer.writerow(row)
        fi.close()
      except Exception as e:
        print(e)

  dataset = pd.read_csv(f'{path}/dataset.csv')
        
  return dataset

In [ ]:
#dataset= create_dataset(emmbed_dim=50, max_words=max_seq_length)
dataset = pd.read_csv('/content/drive/MyDrive/PFA/dataset.csv')

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15594 entries, 0 to 15593
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      15594 non-null  object
 1   text    15594 non-null  object
 2   pos     15594 non-null  object
 3   arg     15594 non-null  object
dtypes: object(4)
memory usage: 487.4+ KB


In [ ]:
t = Tokenizer()
t.fit_on_texts([text for text in dataset['text'].values])
vocab_size = len(t.word_index)+1

In [ ]:
print(vocab_size)

67888


In [ ]:
from keras.preprocessing.sequence import pad_sequences
X = dataset['text'].values
y = dataset['arg'].values

y=[np.array([float(x1) for x1 in x.split()]) for x in y]

X_encode = t.texts_to_sequences(X)
X_padded = pad_sequences(X_encode, maxlen=max_seq_length,padding='post')
y_padded = pad_sequences(y, maxlen=max_seq_length,padding='post')


In [ ]:
X_padded.shape

(15594, 500)

In [ ]:
from keras.utils import np_utils
y_cat = np_utils.to_categorical(y_padded)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_padded,y_cat, test_size=0.2, random_state=42 )

In [ ]:
embedding_matrix = np.zeros((vocab_size,50))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [ ]:
from keras import Sequential

def modelLSTM(input_length,internal_units,output_size,vocab_size,embedding_matrix=embedding_matrix,embedding_dim=50):
    model = Sequential()
    model.add(layers.Embedding(vocab_size,embedding_dim,weights=[embedding_matrix],input_length=input_length,trainable=False))
    model.add(layers.LSTM(input_length=50,units=internal_units,return_sequences=True))
    #model.add(layers.Flatten()) 
    model.add(layers.TimeDistributed(layers.Dense(output_size,activation='softmax')))
    return model


def loss_function(y_true,y_pred):
  return sum(np.float32(y_true)-np.float32(y_pred))/len(np.float32(y_true))

In [ ]:
model = modelLSTM(input_length=max_seq_length,internal_units=64,output_size=y_test.shape[2],vocab_size=vocab_size)
model.build()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 50)           3394400   
                                                                 
 lstm (LSTM)                 (None, 500, 64)           29440     
                                                                 
 time_distributed (TimeDistr  (None, 500, 8)           520       
 ibuted)                                                         
                                                                 
Total params: 3,424,360
Trainable params: 29,960
Non-trainable params: 3,394,400
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
history=model.fit(x=X_train, y=y_train, batch_size=256, epochs=10, validation_data=(X_test, y_test), verbose=1)
loss, accuracy = model.evaluate(X_test,  y_test, verbose=1)
print('Accuracy: %f' % (accuracy*100))

Epoch 1/10
49/49 [==============================] - 10s 46ms/step - loss: 1.4480 - categorical_accuracy: 0.6451 - val_loss: 1.1165 - val_categorical_accuracy: 0.6518
Epoch 2/10
49/49 [==============================] - 2s 33ms/step - loss: 1.0647 - categorical_accuracy: 0.6621 - val_loss: 0.9896 - val_categorical_accuracy: 0.6851
Epoch 3/10
49/49 [==============================] - 2s 35ms/step - loss: 0.9391 - categorical_accuracy: 0.6935 - val_loss: 0.9072 - val_categorical_accuracy: 0.6974
Epoch 4/10
49/49 [==============================] - 2s 34ms/step - loss: 0.8917 - categorical_accuracy: 0.7010 - val_loss: 0.8889 - val_categorical_accuracy: 0.6987
Epoch 5/10
49/49 [==============================] - 2s 36ms/step - loss: 0.8756 - categorical_accuracy: 0.7040 - val_loss: 0.8839 - val_categorical_accuracy: 0.7020
Epoch 6/10
49/49 [==============================] - 2s 35ms/step - loss: 0.8616 - categorical_accuracy: 0.7090 - val_loss: 0.8568 - val_categorical_accuracy: 0.7096
Epoch 7/1

In [ ]:
loss, accuracy = model.evaluate(X_test,  y_test, verbose=1)
print('Accuracy: %f' % (accuracy*100))

98/98 [==============================] - 1s 8ms/step - loss: 0.8320 - categorical_accuracy: 0.7189
Accuracy: 71.892786


In [ ]:
from sklearn.metrics import confusion_matrix, f1_score
ydl_pred = np.argmax(model.predict(X_test), axis=-1)
y_true = np.argmax(y_test,axis=-1)

In [ ]:
cm=[ f1_score(y_t,y_p,average='weighted') for y_t,y_p in zip(y_true,ydl_pred)]

In [ ]:
np.mean(cm)

0.6726880606845136

In [ ]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

In [ ]:
print(ydl_pred[0])

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 4 4 4 4 4 4 4 4 4 4 2 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 5 5 3 3 3 

In [ ]:
y_true[0]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [ ]:
def modelLSTM1(input_length,internal_units,output_size,vocab_size,embedding_matrix=embedding_matrix,embedding_dim=50):
    model = Sequential()
    model.add(layers.Embedding(vocab_size,embedding_dim,weights=[embedding_matrix],input_length=input_length,trainable=False))
    model.add(layers.LSTM(input_length=50,units=internal_units,return_sequences=True))
    #model.add(layers.Flatten()) 
    model.add(layers.TimeDistributed(layers.Dense(output_size,activation='softmax')))
    return model


In [ ]:
!rm -r /content/drive/MyDrive/PFA/treated-data

In [ ]:
!mkdir /content/drive/MyDrive/PFA/treated-data ; cp /content/data/train/* /content/drive/MyDrive/PFA/treated-data